In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

supermarket_df = pd.read_csv("完整数据.csv"
                             ,encoding="Windows-1252")
supermarket_df

观察到Date中Dtype为object，为便于分析，对其进行转化

In [ ]:
supermarket_df["Order Date"] = pd.to_datetime(supermarket_df["Order Date"], format="mixed")
supermarket_df["Ship Date"] = pd.to_datetime(supermarket_df["Ship Date"], format="mixed")
supermarket_df.info()

此时数据类型已转换完成，接下来进行数据清洗

In [ ]:
# 邮编缺失值过多，删除
supermarket_df.drop(columns="Postal Code", axis=0, inplace=True)
# 查看行是否存在重复值
print(f"Duplicated: {supermarket_df.duplicated().sum()}")
# 查看Row ID是否存在重复值
print(f"Row ID Duplicated: {supermarket_df[['Row ID']].duplicated().sum()}")

print(supermarket_df[supermarket_df.isna().any(axis=1)])

# 可视化


## 每月总销售额

## 

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

alt.Chart(supermarket_df).mark_bar().encode(
  x="month(Order Date):T",
  y="sum(Sales):Q",
).properties(
  width=900
).interactive()

## 销售额折线图（按月分）

In [ ]:
alt.Chart(supermarket_df).mark_line().encode(
  x=alt.X("yearmonth(Order Date):T", title="Order Date"),
  y=alt.Y("sum(Sales):Q", title="Sum of Sales"),
).properties(
  width=900
).interactive()

## 各市场订单量排名

In [ ]:
alt.Chart(supermarket_df).mark_bar().encode(
  y=alt.Y(field="Market", sort="-x"),
  x="sum(Sales)",
  color=alt.Color("sum(Sales):Q")
).properties(
  width=600
).interactive()

## 前20销售额最高国家

In [ ]:
alt.Chart(supermarket_df).transform_aggregate(
  sales_sum="sum(Sales)",
  groupby=["Country"]
).transform_window(
    rank='rank(sales_sum)',
    sort=[alt.SortField('sales_sum', order='descending')]
).transform_filter(
    (alt.datum.rank < 20)
).mark_bar(
).encode(
  x=alt.X("sales_sum:Q", title="总销售额"),
  y=alt.Y(field="Country", sort="-x"),
  color=alt.Color("sales_sum:Q", title="总销售额")
).properties(
  width=900
)

## 订单优先级与利润关系

In [ ]:
alt.Chart(supermarket_df).mark_bar(
).encode(
  y=alt.Y("Order Priority",sort="-x"),
  x=alt.X("mean(Profit)"),
  color=alt.Color("mean(Profit)")
).properties(
  width=900
)

## 季度销量图

In [ ]:
alt.Chart(supermarket_df).mark_bar(
).encode(
  x="year(Order Date):T",
  y="sum(Sales):Q",
  column="quarter(Order Date):T",
  color="sum(Sales):Q"
)

## 子分类的平均利润排行

In [ ]:
supermarket_df["Sub-Category"].unique()

alt.Chart(supermarket_df).transform_aggregate(
  avg_profit="mean(Profit)",
  groupby=["Sub-Category"]
).mark_bar(
).encode(
  x=alt.X("avg_profit:Q", title="Mean of Profit"),
  y=alt.Y("Sub-Category", sort="-x"),
  color="avg_profit:Q"
)

# 不同客户类别销售额总和

In [ ]:
a = supermarket_df.copy()
a = a.groupby("Segment")["Sales"].sum().reset_index()
a["percentage"] = (a["Sales"] / a["Sales"].sum() * 100).apply(lambda x: f"{x:.2f}%")
print(a)
pie = alt.Chart(a).mark_arc(outerRadius=160).encode(
  theta=alt.Theta("Sales").stack(True),
  color=alt.Color("Segment:N")
)
text = pie.mark_text(radius=180, size=10).encode(
  text="percentage:N",
)
(pie + text)


# 利润比销售额

In [ ]:
prof_sales = supermarket_df.groupby(supermarket_df["Order Date"].dt.year)[["Sales", "Profit"]].sum().reset_index()
prof_sales["Ratio"] = prof_sales["Profit"] / prof_sales["Sales"]

alt.Chart(prof_sales).mark_bar().encode(
  x=alt.X("Order Date:N", axis=alt.Axis(labelAngle=0)),
  y=alt.Y("Ratio:Q", title="Profit by Sales"),
  color=alt.Color("Ratio:Q")
).properties(
  height=200,
  width=500
)
